In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Normalize, ColorJitter, RandomHorizontalFlip, RandomRotation
from PIL import Image
import os
import numpy as np
import random
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight

# =============================== #
#      Define Class Information   #
# =============================== #

# Simplified class representation using lists instead of namedtuples
class_names = ["background", "film", "basket", "cardboard", "video_tape", "filament", "bag"]
ignore_in_eval = [True, False, False, False, False, False, False]  # Only ignore background
class_colors = [(0, 0, 0), (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]
num_classes = len(class_names)

# =============================== #
#        Define Device            #
# =============================== #

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# =============================== #
#     Set Seed for Reproducibility #
# =============================== #

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

# =============================== #
#       Define Dataset Class      #
# =============================== #

class SegmentationDataset(Dataset):
    def __init__(self, img_dir, mask_dir, image_transform=None, mask_transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.image_transform = image_transform
        self.mask_transform = mask_transform
        
        # Get sorted image and mask filenames
        self.image_filenames = sorted([f for f in os.listdir(img_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        self.mask_filenames = sorted([f for f in os.listdir(mask_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        
        # Verify we found all files
        if len(self.image_filenames) != len(self.mask_filenames):
            raise ValueError(f"Found {len(self.image_filenames)} images but {len(self.mask_filenames)} masks")
            
        print(f"Dataset created with {len(self.image_filenames)} samples")

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_filenames[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_filenames[idx])

        # Load image and mask
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)

        # Ensure identical transformations are applied to both image and mask
        if self.image_transform and self.mask_transform:
            seed = torch.randint(0, 2**32, (1,)).item()
            
            torch.manual_seed(seed)
            random.seed(seed)
            image = self.image_transform(image)

            torch.manual_seed(seed)
            random.seed(seed)
            mask = self.mask_transform(mask)
        elif self.image_transform:
            image = self.image_transform(image)

        # Convert mask to tensor
        mask_tensor = torch.tensor(np.array(mask), dtype=torch.long)

        return image, mask_tensor

# =============================== #
#        Define Transformations   #
# =============================== #

train_image_transform = Compose([
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    RandomHorizontalFlip(p=0.5),
    RandomRotation(degrees=20),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_mask_transform = Compose([
    RandomHorizontalFlip(p=0.5),
    RandomRotation(degrees=20)
])

val_image_transform = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_mask_transform = None  # No transformations for validation masks

# =============================== #
#      Function to generate stratification labels
# =============================== #

def get_labels(dataset):
    """Generate labels for stratified split based on presence of foreground classes"""
    labels = []
    for i in range(len(dataset)):
        _, mask = dataset[i]
        mask_np = mask.numpy()
        # 1 if any foreground class is present, 0 otherwise
        labels.append(1 if np.any(mask_np > 0) else 0)
    return labels

# =============================== #
#        Dataset Setup with Stratified Split
# =============================== #

# Directory paths
img_dir = "./rgb/train"
mask_dir = "./labels_rgb/train"

# Initialize dataset
full_dataset = SegmentationDataset(
    img_dir=img_dir,
    mask_dir=mask_dir,
    image_transform=None,  # No transforms yet
    mask_transform=None
)

# Get stratification labels
print("Generating stratification labels...")
labels = get_labels(full_dataset)
print(f"Found {sum(labels)} images with foreground classes and {len(labels) - sum(labels)} background-only images")

# Perform stratified split
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(split.split(np.zeros(len(labels)), labels))

# Create train and validation subsets
train_dataset = torch.utils.data.Subset(full_dataset, train_idx)
val_dataset = torch.utils.data.Subset(full_dataset, val_idx)

print(f"Split dataset: {len(train_dataset)} training samples, {len(val_dataset)} validation samples")

# Apply appropriate transforms to each subset
train_dataset.dataset.image_transform = train_image_transform
train_dataset.dataset.mask_transform = train_mask_transform
val_dataset.dataset.image_transform = val_image_transform
val_dataset.dataset.mask_transform = val_mask_transform

# DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# =============================== #
#        Define U-Net Model       #
# =============================== #

class UNet(nn.Module):
    def __init__(self, num_classes):
        super(UNet, self).__init__()
        # Load a pretrained ResNet34 as encoder with specific weights version
        self.encoder = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
        
        # Extract different stages of the ResNet
        self.initial = nn.Sequential(*list(self.encoder.children())[:3])
        self.maxpool = list(self.encoder.children())[3]
        self.encoder1 = list(self.encoder.children())[4]
        self.encoder2 = list(self.encoder.children())[5]
        self.encoder3 = list(self.encoder.children())[6]
        self.encoder4 = list(self.encoder.children())[7]

        # Decoder stages
        self.upconv4 = self.up_conv(512, 256)
        self.decoder4 = self.decoder_block(512, 256)
        self.upconv3 = self.up_conv(256, 128)
        self.decoder3 = self.decoder_block(256, 128)
        self.upconv2 = self.up_conv(128, 64)
        self.decoder2 = self.decoder_block(128, 64)
        self.upconv1 = self.up_conv(64, 64)
        self.decoder1 = self.decoder_block(128, 64)
        
        # Final layer for classification
        self.conv_last = nn.Conv2d(64, num_classes, kernel_size=1)
    
    def up_conv(self, in_channels, out_channels):
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def decoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder path
        x0 = self.initial(x)  # Initial conv layers
        x1 = self.maxpool(x0)
        x1 = self.encoder1(x1)  # First encoder block
        x2 = self.encoder2(x1)  # Second encoder block
        x3 = self.encoder3(x2)  # Third encoder block
        x4 = self.encoder4(x3)  # Fourth encoder block
    
        # Decoder path with skip connections
        d4 = self.upconv4(x4)
        d4 = torch.cat((d4, x3), dim=1)  # Skip connection from encoder3
        d4 = self.decoder4(d4)
    
        d3 = self.upconv3(d4)
        d3 = torch.cat((d3, x2), dim=1)  # Skip connection from encoder2
        d3 = self.decoder3(d3)
    
        d2 = self.upconv2(d3)
        d2 = torch.cat((d2, x1), dim=1)  # Skip connection from encoder1
        d2 = self.decoder2(d2)
    
        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, x0), dim=1)  # Skip connection from initial
        d1 = self.decoder1(d1)
    
        # Ensure output size matches input size (256x256)
        output = F.interpolate(d1, size=(256, 256), mode="bilinear", align_corners=True)
        return self.conv_last(output)

# Initialize the model
model = UNet(num_classes=num_classes).to(device)

# =============================== #
#       Compute Class Weights     #
# =============================== #

def compute_class_weights(dataset):
    """Use scikit-learn's compute_class_weight for better class weighting"""
    print("Computing class weights...")
    
    # Collect all mask labels
    mask_flat = []
    num_samples = min(500, len(dataset))  # Sample a subset for efficiency
    indices = random.sample(range(len(dataset)), num_samples)
    
    for idx in tqdm(indices, desc="Collecting mask pixels"):
        _, mask = dataset[idx]
        mask_flat.extend(mask.numpy().flatten())
    
    # Compute balanced class weights
    class_weights = compute_class_weight(
        class_weight="balanced",
        classes=np.arange(num_classes),
        y=mask_flat
    )
    
    # Adjust background weight
    if ignore_in_eval[0]:  # If background is ignored in evaluation
        class_weights[0] *= 0.1  # Reduce background weight
    
    weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
    print("Computed class weights:", weights_tensor)
    
    return weights_tensor

# Compute class weights for the loss function
class_weights = compute_class_weights(train_dataset)

# =============================== #
#      Define Loss Functions      #
# =============================== #

# Define Dice Loss
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, inputs, targets):
        inputs_soft = F.softmax(inputs, dim=1)
        targets_onehot = F.one_hot(targets, num_classes=inputs.shape[1]).permute(0, 3, 1, 2).float()
        intersection = torch.sum(inputs_soft * targets_onehot, dim=(0, 2, 3))
        union = torch.sum(inputs_soft + targets_onehot, dim=(0, 2, 3))
        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

# Define Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, weight=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(weight=weight)
    def forward(self, inputs, targets):
        log_probs = F.log_softmax(inputs, dim=1)
        probs = torch.exp(log_probs)
        loss = (1 - probs) ** self.gamma * self.ce(inputs, targets)
        return loss.mean()

# Define combined loss function (Focal Loss + Dice Loss)
criterion = lambda outputs, targets: 0.5 * FocalLoss(weight=class_weights)(outputs, targets) + 0.5 * DiceLoss()(outputs, targets)

# =============================== #
#       Define Optimizer & LR Scheduler 
# =============================== #

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Define learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',  # Maximize IoU
    factor=0.5,  # Reduce LR by half when plateauing
    patience=5,  # Wait 5 epochs before reducing LR
    min_lr=1e-6  # Minimum LR
)

# =============================== #
#        IoU Calculation         #
# =============================== #

def calculate_iou(preds, masks, num_classes):
    """Calculate IoU for each class and mean IoU."""
    intersection = torch.zeros(num_classes).to(device)
    union = torch.zeros(num_classes).to(device)

    # Count presence of each class for debugging
    class_in_pred = [False] * num_classes
    class_in_target = [False] * num_classes

    for cls in range(num_classes):
        if ignore_in_eval[cls]:
            continue  # Skip background class in IoU calculation

        pred_inds = (preds == cls)
        target_inds = (masks == cls)
        
        # Check if class exists in predictions and targets
        class_in_pred[cls] = pred_inds.any().item()
        class_in_target[cls] = target_inds.any().item()

        intersection[cls] += (pred_inds & target_inds).sum().float()
        union[cls] += (pred_inds | target_inds).sum().float()

    # Calculate IoU for each class
    iou_per_class = []
    for cls in range(num_classes):
        if ignore_in_eval[cls]:
            iou_per_class.append(float('nan'))  # Mark background class as NaN
        else:
            if union[cls] > 0:
                iou = intersection[cls] / union[cls]
                iou_per_class.append(iou.item())
            else:
                # If class is not present in either prediction or ground truth
                iou_per_class.append(float('nan'))

    # Compute mean IoU excluding background and classes not present
    valid_ious = [iou for iou in iou_per_class if not np.isnan(iou)]
    mean_iou = np.mean(valid_ious) if valid_ious else 0.0

    return mean_iou, iou_per_class, class_in_pred, class_in_target

# =============================== #
#        Training Loop            #
# =============================== #

num_epochs = 50
best_iou = 0.0
history = {
    'train_loss': [], 
    'train_iou': [], 
    'val_loss': [], 
    'val_iou': [], 
    'lr': []
}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0.0
    
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    # Training loop
    progress_bar = tqdm(train_loader, desc="Training")
    for images, masks in progress_bar:
        images, masks = images.to(device), masks.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Update metrics
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    
    # Calculate average training loss
    avg_train_loss = total_loss / len(train_loader)
    history['train_loss'].append(avg_train_loss)
    history['lr'].append(optimizer.param_groups[0]['lr'])
    
    # Calculate training IoU
    model.eval()
    train_preds, train_masks = [], []
    with torch.no_grad():
        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            train_preds.append(preds)
            train_masks.append(masks)
    
    train_preds = torch.cat(train_preds, dim=0)
    train_masks = torch.cat(train_masks, dim=0)
    train_mean_iou, train_iou_per_class, _, _ = calculate_iou(train_preds, train_masks, num_classes)
    history['train_iou'].append(train_mean_iou)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Mean IoU: {train_mean_iou:.4f}")
    print("Train IoU per Class:")
    for cls in range(num_classes):
        if not ignore_in_eval[cls]:
            class_iou = train_iou_per_class[cls]
            if not np.isnan(class_iou):
                print(f"  {class_names[cls]}: {class_iou:.4f}")
            else:
                print(f"  {class_names[cls]}: N/A (not present)")
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    val_preds, val_masks_all = [], []
    
    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc="Validation"):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item()
            
            preds = torch.argmax(outputs, dim=1)
            val_preds.append(preds)
            val_masks_all.append(masks)
    
    # Calculate validation metrics
    avg_val_loss = val_loss / len(val_loader)
    history['val_loss'].append(avg_val_loss)
    
    val_preds = torch.cat(val_preds, dim=0)
    val_masks_all = torch.cat(val_masks_all, dim=0)
    val_mean_iou, val_iou_per_class, classes_in_pred, classes_in_gt = calculate_iou(val_preds, val_masks_all, num_classes)
    history['val_iou'].append(val_mean_iou)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Val Loss: {avg_val_loss:.4f}, Val Mean IoU: {val_mean_iou:.4f}")
    print("Validation IoU per Class:")
    for cls in range(num_classes):
        if not ignore_in_eval[cls]:
            class_iou = val_iou_per_class[cls]
            if not np.isnan(class_iou):
                print(f"  {class_names[cls]}: {class_iou:.4f}")
            else:
                print(f"  {class_names[cls]}: N/A (not present)")
    
    # Print class presence information
    print("Classes present in predictions:", [class_names[i] for i in range(num_classes) if classes_in_pred[i]])
    print("Classes present in ground truth:", [class_names[i] for i in range(num_classes) if classes_in_gt[i]])
    
    # Update learning rate based on validation IoU
    scheduler.step(val_mean_iou)
    
    # Save best model
    if val_mean_iou > best_iou:
        print(f"New best IoU achieved: {val_mean_iou:.4f} (previous: {best_iou:.4f})")
        best_iou = val_mean_iou
        torch.save(model.state_dict(), "best_model.pth")
        print("Best model saved!")
    
    print("-" * 50)

# =============================== #
#        Final Evaluation         #
# =============================== #

def evaluate_model(model, loader, dataset_name):
    model.eval()
    total_loss = 0.0
    all_preds, all_masks = [], []

    with torch.no_grad():
        for images, masks in tqdm(loader, desc=f"Evaluating {dataset_name}"):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss.item() * images.size(0)
            preds = torch.argmax(outputs, dim=1)
            all_preds.append(preds)
            all_masks.append(masks)

    all_preds = torch.cat(all_preds, dim=0)
    all_masks = torch.cat(all_masks, dim=0)
    mean_iou, iou_per_class, _, _ = calculate_iou(all_preds, all_masks, num_classes)

    total_loss /= len(loader.dataset)
    print(f"{dataset_name} Results:")
    print(f"Loss: {total_loss:.4f}")
    print(f"Mean IoU: {mean_iou:.4f}")
    print("IoU per Class:")
    for cls in range(num_classes):
        if not ignore_in_eval[cls]:
            class_iou = iou_per_class[cls]
            if not np.isnan(class_iou):
                print(f"  {class_names[cls]}: {class_iou:.4f}")
            else:
                print(f"  {class_names[cls]}: N/A (not present)")
    print("-" * 50)

# Load best model for final evaluation
print("Loading best model for final evaluation...")
model.load_state_dict(torch.load("best_model.pth"))

print("Final Model Evaluation:")
evaluate_model(model, train_loader, "Training Set")
evaluate_model(model, val_loader, "Validation Set")

print("Training and evaluation complete!")

Using device: cuda
Dataset created with 514 samples
Generating stratification labels...
Found 482 images with foreground classes and 32 background-only images
Split dataset: 411 training samples, 103 validation samples
Computing class weights...


Computed class weights: tensor([1.7419e-02, 2.6494e+00, 4.6434e+00, 2.4004e+01, 1.2096e+01, 1.5630e+02,
        1.8675e+00], device='cuda:0')

Epoch 1/50


Training: 100%|██████████| 26/26 [00:06<00:00,  3.78it/s, loss=0.989]


Epoch [1/50], Train Loss: 1.1187, Train Mean IoU: 0.0498
Train IoU per Class:
  film: 0.0933
  basket: 0.0420
  cardboard: 0.0266
  video_tape: 0.1323
  filament: 0.0043
  bag: 0.0005


Validation: 100%|██████████| 7/7 [00:00<00:00, 10.93it/s]


...

Epoch [50/50], Val Loss: 0.7597, Val Mean IoU: 0.4293
Validation IoU per Class:
  film: 0.5570
  basket: 0.6936
  cardboard: 0.3595
  video_tape: 0.2852
  filament: 0.1146
  bag: 0.5658
Classes present in predictions: ['film', 'basket', 'cardboard', 'video_tape', 'filament', 'bag']
Classes present in ground truth: ['film', 'basket', 'cardboard', 'video_tape', 'filament', 'bag']
New best IoU achieved: 0.4293 (previous: 0.4268)
Best model saved!
--------------------------------------------------
Loading best model for final evaluation...
Final Model Evaluation:


Evaluating Training Set: 100%|██████████| 26/26 [00:02<00:00, 12.23it/s]


Training Set Results:
Loss: 0.2238
Mean IoU: 0.6575
IoU per Class:
  film: 0.8850
  basket: 0.8326
  cardboard: 0.8210
  video_tape: 0.3991
  filament: 0.1427
  bag: 0.8646
--------------------------------------------------


Evaluating Validation Set: 100%|██████████| 7/7 [00:00<00:00, 13.87it/s]

Validation Set Results:
Loss: 0.7757
Mean IoU: 0.4293
IoU per Class:
  film: 0.5570
  basket: 0.6936
  cardboard: 0.3595
  video_tape: 0.2852
  filament: 0.1146
  bag: 0.5658
--------------------------------------------------
Training and evaluation complete!
